<a href="https://colab.research.google.com/github/rtajeong/M1/blob/main/gg15_sexpredict_rev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 영문 이름으로 성별 예측

### NLTK(Natural Language Toolkit) 패키지
- NLTK는 교육용으로 개발된 자연어 처리 및 문서 분석용 파이썬 패키지다. 다양한 기능 및 예제를 가지고 있으며 실무 및 연구에서도 
많이 사용된다. NLTK 패키지가 제공하는 주요 기능은 다음과 같다.

 * 말뭉치
 * 토큰 생성
 * 형태소 분석
 * 품사 태깅
 
- 말뭉치
말뭉치(corpus)는 자연어 분석 작업을 위해 만든 샘플 문서 집합을 말한다. 단순히 소설, 신문 등의 문서를 모아놓은 것도 
있지만 품사. 형태소, 등의 보조적 의미를 추가하고 쉬운 분석을 위해 구조적인 형태로 정리해 놓은 것을 포함한다. 
NLTK 패키지의 corpus 서브패키지에서는 다양한 연구용 말뭉치를 제공한다. 이 목록은 전체 corpus의 일부일 뿐이다. 
말뭉치 자료는 설치시에 제공되지 않고 download 명령으로 사용자가 다운로드 받아야 한다. nltk.download('book') 
명령을 실행하면 NLTK 패키지 사용자 설명서에서 요구하는 대부분의 말뭉치를 다운로드 받아준다.

In [ ]:
from nltk.corpus import names          # nltk: national language toolkit (자연어처리 패키지)
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [ ]:
names.words()[:5]

['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi']

In [ ]:
names.words('male.txt')[:5]    # list

['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot']

In [ ]:
# 남자 이름과 여자 이름이 데이터베이스가 나뉘어 있음
labeled_names = [(name, '남자') for name in names.words('male.txt')] +   \
                 [(name, '여자') for name in names.words('female.txt')]   # list
                 
labeled_names[:5], labeled_names[-5:]

([('Aamir', '남자'),
  ('Aaron', '남자'),
  ('Abbey', '남자'),
  ('Abbie', '남자'),
  ('Abbot', '남자')],
 [('Zorine', '여자'),
  ('Zsa Zsa', '여자'),
  ('Zsazsa', '여자'),
  ('Zulema', '여자'),
  ('Zuzana', '여자')])

- 앞에는 남자, 뒤에는 여자의 이름이 들어 있다. 데이터 분석에서 랜덤한 성격을 주기 위해서 이들을 랜덤하게 섞는 작업이 필요하다. 이를 위해서 shuffle() 함수를 사용한다.

In [ ]:
import random
random.shuffle(labeled_names)
labeled_names[:10]

[('Aurelie', '여자'),
 ('Margareta', '여자'),
 ('Marcella', '여자'),
 ('Marion', '남자'),
 ('Chandra', '여자'),
 ('Stormy', '여자'),
 ('Constance', '여자'),
 ('Lizabeth', '여자'),
 ('Rochella', '여자'),
 ('Carmita', '여자')]

In [ ]:
len(labeled_names)

7944

- 이름의 마지막 글자를 얻는 함수 gender_features()라고 만들었다.

In [ ]:
# 이름의 마지막 알파벳을 특성으로 사용 (이게 성별과 관련이 많다고 알려짐)
def gender_features(word):
    return {'last_letter': word[-1]}

gender_features('Sopi')

{'last_letter': 'i'}

In [ ]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
featuresets[0:10]

[({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'n'}, '남자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'y'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'h'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'a'}, '여자')]

In [ ]:
len(featuresets)

7944

In [ ]:
train_set, test_set = featuresets[2000:], featuresets[:2000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.classify(gender_features('Sephipa'))   # predict

'여자'

In [ ]:
classifier.classify(gender_features('Chanho'))

'남자'

In [ ]:
# 분류 모델이 동작을 할 때 어떤 요소들이 중요한 역할을 했는지를 아래와 같이 알아 볼 수 있다. 
# 여기서는 중요한 요소 상위 5개만 출력
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'                여자 : 남자     =     38.0 : 1.0
             last_letter = 'k'                남자 : 여자     =     35.9 : 1.0
             last_letter = 'f'                남자 : 여자     =     12.6 : 1.0
             last_letter = 'm'                남자 : 여자     =     10.9 : 1.0
             last_letter = 'd'                남자 : 여자     =      9.3 : 1.0


In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

0.7575


### 다양한 특성 사용
- 이번에는 새로운 특성을 사용해 본다. 이름의 첫 번째 글자, 이름의 길이, 그리고 각 알파벳이 몇 번 등장했는지를 추가로 특성으로 사용해보겠다. 
- features에 다음과 같이 특성 컬럼 들을 추가했다. 이러한 특성을 얻는 함수를 gender_features2라고 정의했다.

In [ ]:
def gender_features2(name):
    features = {}                                   # dict
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    features["length"] = len(name)
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

gender_features2('Josssshua')

{'first_letter': 'j',
 'last_letter': 'a',
 'length': 9,
 'count(a)': 1,
 'has(a)': True,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 0,
 'has(n)': False,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 4,
 'has(s)': True,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 1,
 'has(u)': True,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [ ]:
labeled_names[0:10]

[('Aurelie', '여자'),
 ('Margareta', '여자'),
 ('Marcella', '여자'),
 ('Marion', '남자'),
 ('Chandra', '여자'),
 ('Stormy', '여자'),
 ('Constance', '여자'),
 ('Lizabeth', '여자'),
 ('Rochella', '여자'),
 ('Carmita', '여자')]

In [ ]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
featuresets[0]

({'first_letter': 'a',
  'last_letter': 'e',
  'length': 7,
  'count(a)': 1,
  'has(a)': True,
  'count(b)': 0,
  'has(b)': False,
  'count(c)': 0,
  'has(c)': False,
  'count(d)': 0,
  'has(d)': False,
  'count(e)': 2,
  'has(e)': True,
  'count(f)': 0,
  'has(f)': False,
  'count(g)': 0,
  'has(g)': False,
  'count(h)': 0,
  'has(h)': False,
  'count(i)': 1,
  'has(i)': True,
  'count(j)': 0,
  'has(j)': False,
  'count(k)': 0,
  'has(k)': False,
  'count(l)': 1,
  'has(l)': True,
  'count(m)': 0,
  'has(m)': False,
  'count(n)': 0,
  'has(n)': False,
  'count(o)': 0,
  'has(o)': False,
  'count(p)': 0,
  'has(p)': False,
  'count(q)': 0,
  'has(q)': False,
  'count(r)': 1,
  'has(r)': True,
  'count(s)': 0,
  'has(s)': False,
  'count(t)': 0,
  'has(t)': False,
  'count(u)': 1,
  'has(u)': True,
  'count(v)': 0,
  'has(v)': False,
  'count(w)': 0,
  'has(w)': False,
  'count(x)': 0,
  'has(x)': False,
  'count(y)': 0,
  'has(y)': False,
  'count(z)': 0,
  'has(z)': False},
 '여자')

In [ ]:
train_set, test_set = featuresets[2000:], featuresets[:2000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.774


In [ ]:
classifier.show_most_informative_features(20)   # last letter 가 가장  중요한 역할을 하는 것을 볼 수 있음

Most Informative Features
             last_letter = 'a'                여자 : 남자     =     38.0 : 1.0
             last_letter = 'k'                남자 : 여자     =     35.9 : 1.0
             last_letter = 'f'                남자 : 여자     =     12.6 : 1.0
             last_letter = 'm'                남자 : 여자     =     10.9 : 1.0
             last_letter = 'd'                남자 : 여자     =      9.3 : 1.0
             last_letter = 'v'                남자 : 여자     =      9.2 : 1.0
             last_letter = 'o'                남자 : 여자     =      7.3 : 1.0
             last_letter = 'z'                남자 : 여자     =      7.2 : 1.0
             last_letter = 'r'                남자 : 여자     =      6.2 : 1.0
                count(a) = 3                  여자 : 남자     =      5.8 : 1.0
             last_letter = 'b'                남자 : 여자     =      4.8 : 1.0
             last_letter = 'g'                남자 : 여자     =      4.6 : 1.0
                count(y) = 2                  여자 : 남자     =      4.6 : 1.0

### 위의 결과를 보면,
- 이번 분류에 가장 영향을 많이 준 요소를 20 개 출력했음.
- 이름의 마지막 글자가 대부분 중요한 역할을 하고 있음 관찰.
- 'a' 가 세 개 포함되면 여성의 비율이 높고, 첫 글자가 'w' 이거나 'w' 의 수가 한 개이면 남성의 비율이 높음.

### 검증 대이터에 대해 성능 평가
- 특성 추출에는 gender_features() 를 사용하고 성능은 아래와 같음.

In [ ]:
print(len(labeled_names))
labeled_names[:5]   # list 

7944


[('Aurelie', '여자'),
 ('Margareta', '여자'),
 ('Marcella', '여자'),
 ('Marion', '남자'),
 ('Chandra', '여자')]

In [ ]:
# 검증 데이터에 대해 성능 평가
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [ ]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set)) 

0.762


In [ ]:
train_set[:20]

[({'last_letter': 'd'}, '남자'),
 ({'last_letter': 'y'}, '남자'),
 ({'last_letter': 'l'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'n'}, '남자'),
 ({'last_letter': 'y'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'k'}, '남자'),
 ({'last_letter': 'e'}, '남자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'n'}, '남자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 's'}, '남자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'e'}, '남자'),
 ({'last_letter': 'o'}, '여자'),
 ({'last_letter': 'i'}, '여자'),
 ({'last_letter': 'o'}, '남자'),
 ({'last_letter': 'a'}, '여자')]

#### 잘못 예측한 내용을 나열해 보자

In [ ]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [ ]:
for (tag, guess, name) in sorted(errors):
    print('correct={:<8s} guess={:<8s} name={:<30}'.format(tag, guess, name))
                                                # < : Left aligned to the remaining space (<, >, ^, =)

correct=남자       guess=여자       name=Aguste                        
correct=남자       guess=여자       name=Andie                         
correct=남자       guess=여자       name=Anthony                       
correct=남자       guess=여자       name=Arie                          
correct=남자       guess=여자       name=Aristotle                     
correct=남자       guess=여자       name=Arvy                          
correct=남자       guess=여자       name=Avi                           
correct=남자       guess=여자       name=Barnaby                       
correct=남자       guess=여자       name=Barnie                        
correct=남자       guess=여자       name=Berkley                       
correct=남자       guess=여자       name=Burnaby                       
correct=남자       guess=여자       name=Charlie                       
correct=남자       guess=여자       name=Claire                        
correct=남자       guess=여자       name=Clancy                        
correct=남자       guess=여자       name=Daffy      

### 마지막 두 글자 사용

In [ ]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

In [ ]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]

In [ ]:
train_set[:10]

[({'suffix1': 'd', 'suffix2': 'rd'}, '남자'),
 ({'suffix1': 'y', 'suffix2': 'ey'}, '남자'),
 ({'suffix1': 'l', 'suffix2': 'el'}, '여자'),
 ({'suffix1': 'a', 'suffix2': 'ma'}, '여자'),
 ({'suffix1': 'n', 'suffix2': 'en'}, '남자'),
 ({'suffix1': 'y', 'suffix2': 'ty'}, '여자'),
 ({'suffix1': 'a', 'suffix2': 'la'}, '여자'),
 ({'suffix1': 'k', 'suffix2': 'ck'}, '남자'),
 ({'suffix1': 'e', 'suffix2': 'ie'}, '남자'),
 ({'suffix1': 'a', 'suffix2': 'ta'}, '여자')]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.771


In [ ]:
# train_set[:20]
train_names[:20]

[('Bernhard', '남자'),
 ('Dabney', '남자'),
 ('Hazel', '여자'),
 ('Guillema', '여자'),
 ('Shelden', '남자'),
 ('Dusty', '여자'),
 ('Zola', '여자'),
 ('Ulrick', '남자'),
 ('Brodie', '남자'),
 ('Trista', '여자'),
 ('Sherwin', '남자'),
 ('Justine', '여자'),
 ('Frans', '남자'),
 ('Thomasina', '여자'),
 ('Nissie', '여자'),
 ('Terence', '남자'),
 ('Theo', '여자'),
 ('Tedi', '여자'),
 ('Alfonso', '남자'),
 ('Eirena', '여자')]

# Exercise

In [ ]:
# for exercise
test = [2,3,4,5,2,2,2]
test.count(2)    # 4
test2 = 'abcdefgggggggggggGgggggggggggg'
test2.lower().count('g')    # 24
test2.count('g')  # 23
test2.count('{}'.format('ggg'))       # same, % 연산자 대신 {} 를 써서 format() 사용함. - 새로운 방식

7